In [44]:
import pickle
import numpy as np
import pandas as pd
import itertools
from enum import Enum
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.discriminant_analysis import (
    LinearDiscriminantAnalysis,
    QuadraticDiscriminantAnalysis,
)
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import (
    f_classif,
    mutual_info_classif,
    SelectKBest,
    RFE,
    RFECV,
    SelectFpr,
    SelectFdr,
    SelectFwe,
)
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import HistGradientBoostingClassifier, GradientBoostingClassifier
from sklearn.metrics import accuracy_score, precision_score
import time
RESULTS_COLUMNS = [
    "score",
    "numberOfTruePositives",
    "accuracy",
    "precision",
    "numberOfFeatures",
    "model",
    "model_parameters",
    "feature_selector",
    "selector_parameters",
]

In [45]:
def namestr(obj, namespace) -> str:
    """Get name of a variable as a string"""
    return [name for name in namespace if namespace[name] is obj]

In [46]:
def generateParameters(arrays, namespace):
    """Takes any number of arrays and returns an array of
    dictionaries with keys as array names and values, all possible combinations

    namespace in general should be globals() or locals()
    """

    allCombinations = list(itertools.product(*arrays))

    allCombinations = [list(elem) for elem in allCombinations]

    arrayNames = []

    for array in arrays:
        arrayNames.append(namestr(array, namespace)[0])

    return [dict(zip(arrayNames, value)) for value in allCombinations]

In [47]:
class ModelType(Enum):
    """Available classifiers"""

    LDA = 0

    QDA = 1

    DecisionTree = 2

    KNN = 3

    SVC = 4

    GradientBoosting = 5

    HistGradientBoosting = 6

    MLPClassifier = 7

In [48]:
from sklearn.neural_network import MLPClassifier


def getModel(modelType, arguments):
    """Returns a classifier that can fit() and predict()"""

    match modelType:

        case ModelType.LDA:

            return LinearDiscriminantAnalysis(**arguments)

        case ModelType.QDA:

            return QuadraticDiscriminantAnalysis(**arguments)

        case ModelType.DecisionTree:

            return DecisionTreeClassifier(**arguments)

        case ModelType.KNN:

            return KNeighborsClassifier(**arguments)

        case ModelType.SVC:

            return SVC(**arguments)

        case ModelType.GradientBoosting:

            return GradientBoostingClassifier(**arguments)

        case ModelType.HistGradientBoosting:

            return HistGradientBoostingClassifier(**arguments)
        
        case ModelType.MLPClassifier:
                
            return MLPClassifier(**arguments)

In [49]:
class FeatureSelectorType(Enum):
    """Available Feature Selectors that can fit_transform() a dataset"""

    NoFeatureSelection = 0

    KBest = 1

    RFE = 2

    RFECV = 3

    FPR = 4

    FDR = 5

    FWE = 6

    FPCA = 7

In [50]:
class NoFeatureSelection:
    """Wrapper class for no feature selection, exposes
    fit_transform method that returns unchanged X
    """

    def fit_transform(self, X, y):

        return X

In [51]:
from sklearn.decomposition import PCA


def getFeatureSelector(selectorType, arguments):
    """Returns a feature selector that can fit_transform()"""

    match selectorType:

        case FeatureSelectorType.NoFeatureSelection:

            return NoFeatureSelection()

        case FeatureSelectorType.KBest:

            return SelectKBest(**arguments)
        

In [52]:
def getTotalNoOfExperiments(models, featureSelectors):
    modelCount = 0
    selectorCount = 0
    for model in models:
        modelCount += len(model["parameters"])
    for featureSelector in featureSelectors:
        selectorCount += len(featureSelector["parameters"])
    return modelCount * selectorCount

In [53]:
def getScore(y_true, y_pred, featuresUsed):
    """Get score based on y_true, y_pred and number of feature used

    Scoring function, based on which, the best model is selected.
    Score is calculated according to task description: +10 points for each
    correctly classified positive class, -200 points for each feature used
    """

    score = 0

    correct = 0

    for i, y in enumerate(y_true):
        if y == 1 and y == y_pred[i]:
            correct += 1

    score = 10 * correct - 200 * featuresUsed
    return correct, score

In [54]:
def performExperiment(X_train, y_train, X_test, y_test, model):
    """Returns a score for given model and provided data"""

    model.fit(X_train, y_train)

    y_pred = model.predict(X_test)

    numberOfFeatures = len(X_train[0])

    accuracy = accuracy_score(y_test, y_pred)

    correct, score = getScore(y_test, y_pred, numberOfFeatures)
    # finalResult = getScore(y_test, y_pred, model["classification"].n_features_in_)

    #

    # Take only 1000 of the highest class 1 probabilities

    # TODO: Turn it into a dataframe and keep track of the indexes

    # finalResult = np.sort(result[:, 1])[::-1][:1000]

    return correct, score, accuracy, numberOfFeatures, precision_score(y_test, y_pred)

In [55]:
def conductExperiments(models, featureSelectors, X_orig, y_orig):
    """Collects score results for all provided models and feature selectors on given dataset

    Parameters:

    models: array in a format: [{"model":"model name","parameters":[{paramters object}]}]

    featureSelectors: array in a format: [{"model":"model name","parameters":[{paramters object}]}]


    """
    results = []
    totalNumberOfExperiments = getTotalNoOfExperiments(models, featureSelectors)
    experimentCount = 0

    # scaler = StandardScaler()
    # X_scaled = scaler.fit_transform(X_orig, y_orig)
    for featureSelector in featureSelectors:
        for featureSelectorParameters in featureSelector["parameters"]:
            try:
                startFeatureSelector = time.time()
                selector = getFeatureSelector(
                    featureSelector["model"], featureSelectorParameters
                )

                X_new = selector.fit_transform(X_orig, y_orig)

                endFeatureSelector = time.time()

                if len(X_new[0]) > 1:
                    X_split_train, X_split_test, y_split_train, y_split_test = (
                        train_test_split(X_new, y_orig, test_size=0.33, random_state=42)
                    )
                    # print(len(y_split_test[y_split_test==1]))
                    for model in models:
                        for modelParameters in model["parameters"]:

                            # clf = Pipeline(
                            #     [
                            #         ("scaling", StandardScaler()),
                            #         (
                            #             "feature_selection",
                            #             getFeatureSelector(
                            #                 featureSelector["model"], featureSelectorParameters
                            #             ),
                            #         ),
                            #         (
                            #             "classification",
                            #             getModel(model["model"], modelParameters),
                            #         ),
                            #     ]
                            # )
                            try:
                                # X_new = selector.fit_transform(X_orig, y_orig)
                                startModel = time.time()
                                (
                                    correct,
                                    score,
                                    accuracy,
                                    numberOfFeatures,
                                    precision,
                                ) = performExperiment(
                                    X_train=X_split_train,
                                    y_train=y_split_train,
                                    X_test=X_split_test,
                                    y_test=y_split_test,
                                    model=getModel(model["model"], modelParameters),
                                )

                                results.append(
                                    [
                                        score,
                                        correct,
                                        accuracy,
                                        precision,
                                        numberOfFeatures,
                                        model["model"].name,
                                        modelParameters,
                                        featureSelector["model"].name,
                                        featureSelectorParameters,
                                    ]
                                )
                                endModel = time.time()
                                experimentCount += 1
                                print(
                                    "Performed Experiment",
                                    str(experimentCount)
                                    + "/"
                                    + str(totalNumberOfExperiments)
                                    + "(approx)",
                                    "took (s):",
                                    "model:",
                                    str(round(endModel - startModel, 2)),
                                    "selector",
                                    str(
                                        round(
                                            endFeatureSelector - startFeatureSelector, 2
                                        )
                                    ),
                                    "with:",
                                    featureSelector["model"],
                                    featureSelectorParameters,
                                    model["model"],
                                    modelParameters,
                                )
                            except Exception as e:
                                print(
                                    "!!!Experiment failed for:",
                                    featureSelector["model"],
                                    featureSelectorParameters,
                                    model["model"],
                                    modelParameters,
                                    str(e),
                                )

                else:
                    print(
                        "!!!",
                        featureSelector["model"],
                        "produced 1 or fewer features with parameters:",
                        featureSelectorParameters,
                    )
            except Exception as e:
                print(
                    "!!!Experiment failed for:",
                    featureSelector["model"],
                    featureSelectorParameters,
                    model["model"],
                    modelParameters,
                    str(e),
                )
    return results

In [56]:
with open("./Dataset/x_test.txt") as file:
    X_test = [[float(digit) for digit in line.split()] for line in file]


with open("./Dataset/x_train.txt") as file:
    X_train = [[float(digit) for digit in line.split()] for line in file]


with open("./Dataset/y_train.txt") as file:
    y_train = [[float(digit) for digit in line.split()] for line in file]

In [57]:
print("X_test datapoints:", len(X_test), " features:", len(X_test[0]))
print("X_train datapoints:", len(X_train), " features:", len(X_train[0]))
print("y_train datapoints:", len(y_train))

X_test datapoints: 5000  features: 500
X_train datapoints: 5000  features: 500
y_train datapoints: 5000


In [93]:
RESULTS_FILENAME = "./Results/results-salveen"

In [94]:

# MLPClassifier
solver = ["adam", "sgd"]
activation = ["relu", "logistic", "tanh"]
hidden_layer_sizes = [(100,), (200,), (300,)]
mlpParameters = generateParameters([solver, activation, hidden_layer_sizes], globals())

models = [
    {"model": ModelType.MLPClassifier, "parameters": mlpParameters},
]

In [95]:

k = [2, 5, 10, 20, 40, 80]
score_func = [f_classif, mutual_info_classif]
kBestParameters = generateParameters([score_func, k], globals())

pcaParameters = generateParameters([k], globals())

featureSelectors = [
    {"model": FeatureSelectorType.NoFeatureSelection, "parameters": [{}]},
    {"model": FeatureSelectorType.KBest, "parameters": kBestParameters},
]

# Experiment

In [96]:
y_train_ravel = np.ravel(y_train, order="C")
y_train_ravel = y_train_ravel.astype(int)

In [97]:
%%time
results = conductExperiments(
    models=models,
    featureSelectors=featureSelectors,
    X_orig=X_train,
    y_orig=y_train_ravel,
)
with open(RESULTS_FILENAME, "wb") as f:
    pickle.dump(results, f)

Performed Experiment 1/234(approx) took (s): model: 5.74 selector 0.0 with: FeatureSelectorType.NoFeatureSelection {} ModelType.MLPClassifier {'solver': 'adam', 'activation': 'relu', 'hidden_layer_sizes': (100,)}
Performed Experiment 2/234(approx) took (s): model: 5.58 selector 0.0 with: FeatureSelectorType.NoFeatureSelection {} ModelType.MLPClassifier {'solver': 'adam', 'activation': 'relu', 'hidden_layer_sizes': (200,)}
Performed Experiment 3/234(approx) took (s): model: 6.79 selector 0.0 with: FeatureSelectorType.NoFeatureSelection {} ModelType.MLPClassifier {'solver': 'adam', 'activation': 'relu', 'hidden_layer_sizes': (300,)}


/opt/anaconda3/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


Performed Experiment 4/234(approx) took (s): model: 6.67 selector 0.0 with: FeatureSelectorType.NoFeatureSelection {} ModelType.MLPClassifier {'solver': 'adam', 'activation': 'logistic', 'hidden_layer_sizes': (100,)}
Performed Experiment 5/234(approx) took (s): model: 9.02 selector 0.0 with: FeatureSelectorType.NoFeatureSelection {} ModelType.MLPClassifier {'solver': 'adam', 'activation': 'logistic', 'hidden_layer_sizes': (200,)}
Performed Experiment 6/234(approx) took (s): model: 9.32 selector 0.0 with: FeatureSelectorType.NoFeatureSelection {} ModelType.MLPClassifier {'solver': 'adam', 'activation': 'logistic', 'hidden_layer_sizes': (300,)}
Performed Experiment 7/234(approx) took (s): model: 5.59 selector 0.0 with: FeatureSelectorType.NoFeatureSelection {} ModelType.MLPClassifier {'solver': 'adam', 'activation': 'tanh', 'hidden_layer_sizes': (100,)}
Performed Experiment 8/234(approx) took (s): model: 5.92 selector 0.0 with: FeatureSelectorType.NoFeatureSelection {} ModelType.MLPClass

/opt/anaconda3/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


Performed Experiment 10/234(approx) took (s): model: 5.68 selector 0.0 with: FeatureSelectorType.NoFeatureSelection {} ModelType.MLPClassifier {'solver': 'sgd', 'activation': 'relu', 'hidden_layer_sizes': (100,)}


...

/opt/anaconda3/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


Performed Experiment 232/234(approx) took (s): model: 3.33 selector 3.36 with: FeatureSelectorType.KBest {'score_func': <function mutual_info_classif at 0x17593f100>, 'k': 80} ModelType.MLPClassifier {'solver': 'sgd', 'activation': 'tanh', 'hidden_layer_sizes': (100,)}


/opt/anaconda3/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


Performed Experiment 233/234(approx) took (s): model: 3.36 selector 3.36 with: FeatureSelectorType.KBest {'score_func': <function mutual_info_classif at 0x17593f100>, 'k': 80} ModelType.MLPClassifier {'solver': 'sgd', 'activation': 'tanh', 'hidden_layer_sizes': (200,)}
Performed Experiment 234/234(approx) took (s): model: 4.99 selector 3.36 with: FeatureSelectorType.KBest {'score_func': <function mutual_info_classif at 0x17593f100>, 'k': 80} ModelType.MLPClassifier {'solver': 'sgd', 'activation': 'tanh', 'hidden_layer_sizes': (300,)}
CPU times: user 1h 25min 1s, sys: 4min 6s, total: 1h 29min 8s
Wall time: 8min 48s


/opt/anaconda3/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


In [98]:
with open(RESULTS_FILENAME, "rb") as input_file:
    results = pickle.load(input_file)

In [99]:
resultsDf = pd.DataFrame(
    results,
    columns=RESULTS_COLUMNS,
)

In [100]:
RESULTS_FILENAME = "./Results/results-salveen"
with open(RESULTS_FILENAME, "rb") as input_file:
    results = pickle.load(input_file)
resultsDf = pd.DataFrame(
    results,
    columns=RESULTS_COLUMNS,
)
resultsDf = resultsDf.sort_values(by=["score"], ascending=False)
resultsDf

,score,numberOfTruePositives,accuracy,precision,numberOfFeatures,model,model_parameters,feature_selector,selector_parameters
139,7500,790,0.490909,0.492519,2,MLPClassifier,"{'solver': 'sgd', 'activation': 'logistic', 'h...",KBest,{'score_func': <function mutual_info_classif a...
140,7460,786,0.493939,0.494029,2,MLPClassifier,"{'solver': 'sgd', 'activation': 'logistic', 'h...",KBest,{'score_func': <function mutual_info_classif a...
26,7360,776,0.498182,0.496164,2,MLPClassifier,"{'solver': 'adam', 'activation': 'tanh', 'hidd...",KBest,{'score_func': <function f_classif at 0x17591b...
129,7130,753,0.491515,0.492479,2,MLPClassifier,"{'solver': 'adam', 'activation': 'logistic', '...",KBest,{'score_func': <function mutual_info_classif a...
27,6100,650,0.519394,0.509005,2,MLPClassifier,"{'solver': 'sgd', 'activation': 'relu', 'hidde...",KBest,{'score_func': <function f_classif at 0x17591b...
...,...,...,...,...,...,...,...,...,...
16,-96070,393,0.503636,0.498099,500,MLPClassifier,"{'solver': 'sgd', 'activation': 'tanh', 'hidde...",NoFeatureSelection,{}
8,-96150,385,0.488485,0.482456,500,MLPClassifier,"{'solver': 'adam', 'activation': 'tanh', 'hidd...",NoFeatureSelection,{}
17,-96250,375,0.489091,0.482625,500,MLPClassifier,"{'solver': 'sgd', 'activation': 'tanh', 'hidde...",NoFeatureSelection,{}
10,-96260,374,0.493939,0.487614,500,MLPClassifier,"{'solver': 'sgd', 'activation': 'relu', 'hidde...",NoFeatureSelection,{}


139    {'solver': 'sgd', 'activation': 'logistic', 'h...
Name: model_parameters, dtype: object